In [ ]:
# Importações básicas
import numpy as np
import pandas as pd
# Machine Learning
from sklearn import datasets, tree, model_selection
# Visualização
import matplotlib.pyplot as plt

# Funções
def classificar (decision_tree, atributos, target, param_grid):
  grid_search = model_selection.GridSearchCV(decision_tree, param_grid, cv=5)
  grid_search.fit (atributos, target)
  results = grid_search.cv_results_
  best_estimator = grid_search.best_estimator_
  return results, best_estimator

def imprimir_resultados (results, best_estimator, msg=""):
  print (f"Tempo médio de Teste     {msg}: {results['mean_score_time']}")
  print (f"Acurácia média de Teste  {msg}: {results['mean_test_score']}")
  print (f"Profundidade máxima      {msg}: {best_estimator.tree_.max_depth}")

def gerar_coords (atributos, best_estimator):
  y_min = atributos[:, 1].min()
  y_max = atributos[:, 1].max()

  x_min = atributos[:, 0].min()
  x_max = atributos[:, 0].max()

  feature = best_estimator.tree_.feature
  threshold = best_estimator.tree_.threshold

  coords = []
  for i in range (len(feature)):
    if feature[i] == 0:
      # traçar uma linha vertical
      x0 = threshold[i]
      y0 = y_min
      x1 = threshold[i]
      y1 = y_max
      coords.append (((x0, x1), (y0, y1)))
    elif feature[i] == 1:
      # traçar uma linha horizontal
      x0 = x_min
      y0 = threshold[i]
      x1 = x_max
      y1 = threshold[i]
      coords.append (((x0, x1), (y0, y1)))
  return coords

# Código Principal (main)
atributos, target = datasets.make_blobs (n_features=2, centers=2, n_samples=100, cluster_std=[10, 10])

# MODELO BASELINE (SEM PODA)
param_grid = {'criterion': ['gini']}
results, best_estimator = classificar (tree.DecisionTreeClassifier(), atributos, target, param_grid)

# MODELO COM PODA
param_grid2 = {"max_depth": [2, 4, 6, 8, 10, 12]}
results2, best_estimator2 = classificar (tree.DecisionTreeClassifier(), atributos, target, param_grid2)

#IMPRIMIR RESULTADOS
imprimir_resultados(results, best_estimator, 'BASELINE')
imprimir_resultados(results2, best_estimator2, 'COM PODA')

#GERAR COORDENADAS
coords = gerar_coords(atributos, best_estimator)
coords2 = gerar_coords(atributos, best_estimator2)

# y_min = atributos[:, 1].min()
# y_max = atributos[:, 1].max()

# x_min = atributos[:, 0].min()
# x_max = atributos[:, 0].max()

# feature = best_estimator.tree_.feature
# threshold = best_estimator.tree_.threshold

# coords = []
# for i in range (len(feature)):
#   if feature[i] == 0:
#     # traçar uma linha vertical
#     x0 = threshold[i]
#     y0 = y_min
#     x1 = threshold[i]
#     y1 = y_max
#     coords.append (((x0, x1), (y0, y1)))
#   elif feature[i] == 1:
#     # traçar uma linha horizontal
#     x0 = x_min
#     y0 = threshold[i]
#     x1 = x_max
#     y1 = threshold[i]
#     coords.append (((x0, x1), (y0, y1)))

#print (f"coords: {coords}")

#GERAR GRÁFICO
plt.scatter (atributos[:, 0][target==0], atributos[:, 1][target==0], color='blue')
plt.scatter (atributos[:, 0][target==1], atributos[:, 1][target==1], color='red')
for coord in coords:
  plt.plot (coord[0], coord[1], color='black')
plt.show()

plt.scatter (atributos[:, 0][target==0], atributos[:, 1][target==0], color='blue')
plt.scatter (atributos[:, 0][target==1], atributos[:, 1][target==1], color='red')
for coord in coords2:
  plt.plot (coord[0], coord[1], color='black')
plt.show()

# whole

In [ ]:
import pandas as pd
import requests
from io import StringIO
import matplotlib.pyplot as plt
from sklearn import preprocessing, model_selection, decomposition, tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

# ==============================
# 0. Download dos dados Sonar
# ==============================
def baixar_csv_github(url):
    print(f"Baixando arquivo de {url}...")
    if 'github.com' in url and '/blob/' in url:
        raw_url = url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
    else:
        raw_url = url
    
    response = requests.get(raw_url)
    if response.status_code == 200:
        return pd.read_csv(StringIO(response.text))
    else:
        raise Exception(f"Erro ao baixar arquivo: {response.status_code}")

url_sonar = "https://github.com/professortiagoinfnet/inteligencia_artificial/blob/main/sonar_dataset.csv"
df = baixar_csv_github(url_sonar)

X = df.iloc[:, :-1]
y = preprocessing.LabelEncoder().fit_transform(df.iloc[:, -1])

# ==============================
# Classe de avaliação (já existia no seu código)
# ==============================
class ClassificaBinario:
    @staticmethod
    def calcula_precisao_recall_f1score(y_true, y_pred):
        acuracia = metrics.accuracy_score(y_true, y_pred)
        precisao = metrics.precision_score(y_true, y_pred, zero_division=0)
        recall = metrics.recall_score(y_true, y_pred)
        f1score = metrics.f1_score(y_true, y_pred)
        return acuracia, precisao, recall, f1score

    @staticmethod
    def calcula_acuracia_especificidade(y_true, y_pred):
        acuracia = metrics.accuracy_score(y_true, y_pred)
        mc = metrics.confusion_matrix(y_true, y_pred)
        ((TN, FP), (FN, TP)) = mc
        especificidade = TN / (TN + FP)
        sensibilidade = TP / (TP + FN)
        return acuracia, especificidade, sensibilidade

    @staticmethod
    def plotar_roc(y_true, y_prob, titulo="Curva ROC"):
        fpr, tpr, _ = metrics.roc_curve(y_true, y_prob)
        roc_auc = metrics.auc(fpr, tpr)
        plt.figure(figsize=(7, 5))
        plt.plot(fpr, tpr, color="blue", label=f"ROC Curve (AUC = {roc_auc:.2f})")
        plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
        plt.xlabel("Falso Positivo")
        plt.ylabel("Verdadeiro Positivo")
        plt.title(titulo)
        plt.legend(loc="lower right")
        plt.show()

# ==============================
# 1. PCA aplicado ao KNN
# ==============================
pca = decomposition.PCA(n_components=0.95)  # mantém 95% da variância
X_pca = pca.fit_transform(X)

treino_a, teste_a, treino_c, teste_c = train_test_split(X_pca, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(treino_a, treino_c)
previstos = knn.predict(teste_a)

print("\n=== Avaliação do KNN com PCA (95% variância) ===")
acuracia, precisao, recall, f1score = ClassificaBinario.calcula_precisao_recall_f1score(teste_c, previstos)
_, especificidade, _ = ClassificaBinario.calcula_acuracia_especificidade(teste_c, previstos)

print("Acurácia       :", round(acuracia, 4))
print("Precisão      :", round(precisao, 4))
print("Recall        :", round(recall, 4))
print("F1-Score      :", round(f1score, 4))
print("Especificidade:", round(especificidade, 4))

probabilidades = knn.predict_proba(teste_a)[:, 1]
ClassificaBinario.plotar_roc(teste_c, probabilidades, titulo="Curva ROC - KNN com PCA")

# ==============================
# 2. Árvore de Decisão
# ==============================
treino_a, teste_a, treino_c, teste_c = train_test_split(X, y, test_size=0.2, random_state=42)

decision_tree = tree.DecisionTreeClassifier(random_state=42)
decision_tree.fit(treino_a, treino_c)
previstos = decision_tree.predict(teste_a)

print("\n=== Avaliação da Árvore de Decisão (sem ajuste) ===")
acuracia, precisao, recall, f1score = ClassificaBinario.calcula_precisao_recall_f1score(teste_c, previstos)
_, especificidade, _ = ClassificaBinario.calcula_acuracia_especificidade(teste_c, previstos)

print("Acurácia       :", round(acuracia, 4))
print("Precisão      :", round(precisao, 4))
print("Recall        :", round(recall, 4))
print("F1-Score      :", round(f1score, 4))
print("Especificidade:", round(especificidade, 4))

probabilidades = decision_tree.predict_proba(teste_a)[:, 1]
ClassificaBinario.plotar_roc(teste_c, probabilidades, titulo="Curva ROC - Árvore de Decisão")

# ==============================
# 3. GridSearch na Árvore de Decisão
# ==============================
param_grid = {
    "criterion": ["gini", "entropy", "log_loss"],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=tree.DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1
)

grid_search.fit(X, y)

print("\n=== Melhores hiperparâmetros encontrados ===")
print(grid_search.best_params_)
print("Melhor score:", grid_search.best_score_)
